#### PARTE 1

In [144]:
from neo4j import GraphDatabase

# Preparamos la sesión para acceder a neo4j
driver = GraphDatabase.driver('neo4j://localhost:7687', auth=("neo4j", "BDII2023"))
session = driver.session()

In [153]:
def search_monsters(room_name):
    query = """
    MATCH (r:Room {room_name: $room_name}) - [:CONTAINS] - (m:Monster) 
    MATCH (r2:Room) - [:CONTAINS] - (m)
    MATCH (r2) - [:CONTAINS] - (m2:Monster)
    RETURN DISTINCT m, m2
    """  

    results = session.run(query, room_name=room_name).data()
    monsters_dict = {}
    
    # Iterar sobre los resultados y construir el diccionario
    for record in results:
        m = (record['m']['name'], record['m']['monsters_id'])  # Clave como tupla de nombre e id
        m2 = {'name': record['m2']['name'], 'monsters_id': record['m2']['monsters_id']}  # Valor como diccionario con nombre e id
        
        # Si m no está en el diccionario, agregarlo con una lista vacía como valor
        if m not in monsters_dict:
            monsters_dict[m] = []
        
        # Agregar m2 a la lista correspondiente a m
        monsters_dict[m].append(m2)
        
    if monsters_dict == {}:
        print('Esta sala no tiene monstruos')
    else:      
        return monsters_dict
    

In [154]:
search_monsters('worried workshop ')

Esta sala no tiene monstruos


In [155]:
search_monsters('storage room of degenerates')

{('gargoyle', 1): [{'name': 'gargoyle', 'monsters_id': 1},
  {'name': 'vampire spawn', 'monsters_id': 6},
  {'name': 'doppelganger', 'monsters_id': 0},
  {'name': 'wererat', 'monsters_id': 2},
  {'name': 'revenant', 'monsters_id': 4},
  {'name': 'thug', 'monsters_id': 13},
  {'name': 'assassin', 'monsters_id': 9}],
 ('wererat', 2): [{'name': 'wererat', 'monsters_id': 2},
  {'name': 'vampire spawn', 'monsters_id': 6},
  {'name': 'assassin', 'monsters_id': 9},
  {'name': 'gargoyle', 'monsters_id': 1},
  {'name': 'revenant', 'monsters_id': 4},
  {'name': 'doppelganger', 'monsters_id': 0},
  {'name': 'thug', 'monsters_id': 13}],
 ('revenant', 4): [{'name': 'revenant', 'monsters_id': 4},
  {'name': 'wererat', 'monsters_id': 2},
  {'name': 'gargoyle', 'monsters_id': 1},
  {'name': 'doppelganger', 'monsters_id': 0},
  {'name': 'vampire spawn', 'monsters_id': 6}],
 ('goblin boss', 143): [{'name': 'goblin boss', 'monsters_id': 143},
  {'name': 'weretiger', 'monsters_id': 163},
  {'name': 'scare

#### PARTE 2

In [133]:
from graphdatascience import GraphDataScience
import pandas as pd

# configure connection
gds = GraphDataScience("neo4j://localhost:7687", auth=("neo4j", "BDII2023"))
print(f"GDS version: {gds.version()}")

GDS version: 2.6.5


In [147]:
def monster_recommendations(room_name):
    node_query = """
    MATCH (r1:Room)-[:CONTAINS]->(m1:Monster)<-[:CONTAINS]-(r2:Room)-[:CONTAINS]->(m2:Monster)
    RETURN DISTINCT id(m1) as id
    """
    edge_query = """
    MATCH (r1:Room)-[:CONTAINS]->(m1:Monster)<-[:CONTAINS]-(r2:Room)-[:CONTAINS]->(m2:Monster)
    RETURN DISTINCT id(m1) as source, id(m2) as target, 1 as weight
    """

    with gds.graph.project.cypher("monster_coappearances", node_query, edge_query) as g_temp:
        q_source = """
            MATCH (:Room {room_name: $room_name})-[:CONTAINS]->(m:Monster) 
            RETURN id(m) as sources
        """
        sources = gds.run_cypher(q_source, params={"room_name": room_name}).iloc[:, 0]
        # print(sources)
        result = gds.pageRank.stream(g_temp, sourceNodes=sources, relationshipWeightProperty="weight")
        result = result.query("score > 0")

        nodes = result.nodeId.to_list()
        q = """
        MATCH (:Room {room_name: $room_name})-[:CONTAINS]->(m:Monster) WITH collect(m) as sources
        MATCH (m:Monster) WHERE id(m) IN $nodes AND NOT(m in sources) 
        RETURN id(m) AS nodeId, m.name AS monster
        """
        df = gds.run_cypher(q, params={"room_name": room_name, "nodes": nodes})

        recommendation = pd.merge(result, df, how="left", left_on="nodeId", right_on="nodeId").dropna().sort_values("score", ascending=False).head(5)

    return recommendation

In [148]:
monster_recommendations(room_name='storage room of degenerates')

,nodeId,score,monster
0,11211,0.460563,doppelganger
4,11217,0.326443,vampire spawn
5,11220,0.231330,assassin
6,11224,0.231330,thug
63,11399,0.111986,scarecrow
